# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [19]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
whiteData = data[data.race=='w']
blackData = data[data.race=='b']

print(len(data))
print(len(whiteData))
print(len(blackData))

4870
2435
2435


In [23]:
# number of callbacks for black-sounding names and column names in data frame
sum(data[data.race=='b'].call)
print(data.columns.values)

['id' 'ad' 'education' 'ofjobs' 'yearsexp' 'honors' 'volunteer' 'military'
 'empholes' 'occupspecific' 'occupbroad' 'workinschool' 'email'
 'computerskills' 'specialskills' 'firstname' 'sex' 'race' 'h' 'l' 'call'
 'city' 'kind' 'adid' 'fracblack' 'fracwhite' 'lmedhhinc' 'fracdropout'
 'fraccolp' 'linc' 'col' 'expminreq' 'schoolreq' 'eoe' 'parent_sales'
 'parent_emp' 'branch_sales' 'branch_emp' 'fed' 'fracblack_empzip'
 'fracwhite_empzip' 'lmedhhinc_empzip' 'fracdropout_empzip'
 'fraccolp_empzip' 'linc_empzip' 'manager' 'supervisor' 'secretary'
 'offsupport' 'salesrep' 'retailsales' 'req' 'expreq' 'comreq' 'educreq'
 'compreq' 'orgreq' 'manuf' 'transcom' 'bankreal' 'trade' 'busservice'
 'othservice' 'missind' 'ownership']


In [21]:
whiteData.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
7,b,1,3,4,21,0,1,0,1,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
8,b,1,4,3,3,0,0,0,0,316,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
9,b,1,4,2,6,0,1,0,0,263,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private


In [22]:
blackData.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
5,b,1,4,2,6,1,0,0,0,266,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
6,b,1,4,2,5,0,1,0,0,13,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private


1: We have 4870 samples to look at. The samples are evenly divided between white and black data frames. They are also randomly assigned to these categories and submitted to employers accordingly, which causes the two samples to be independent of each other within and between each other as well. Thus we can use the CLT. We will use the two-sample t-test here.

2: Ho: white callbacks = black callbacks
   Ha: white callbacks != black callbacks

3: Let us compute a 95% confidence interval as well as a p-value of the related test.

In [29]:
import statsmodels.stats.proportion as prop

whiteProportion = len(whiteData[whiteData.call == 1])/(1.0 * len(whiteData))
blackProportion = len(blackData[blackData.call == 1])/(1.0 * len(blackData))

print(whiteProportion)
print(blackProportion)


0.0965092402464
0.064476386037


In [44]:
arr = np.array([len(whiteData[whiteData.call == 1]), len(blackData[blackData.call == 1])])
zstat, pval = prop.proportions_ztest(arr, nobs=2435, value=0, alternative='two-sided')
print(pval)

1.70048622634e-05


This shows that the difference between the proportions is significant. Compute margin of error and 95% confidence interval as well.

In [40]:
ci_low, ci_high = prop.proportion_confint(arr, nobs=2435, alpha=.05, method='normal')

In [42]:
print(ci_low)
print(ci_high)

print(whiteProportion - ci_low[0])
print(blackProportion - ci_low[1])

[ 0.08478067  0.05472141]
[ 0.10823781  0.07423136]
0.01172856595
0.00975497877459


Margin of error for whites: .01173
Margin of error for blacks: .009755

4: According to the results of the hypothesis test, there is a significant difference between callback rates for white sounding resumes vs black sounding resumes. The p-value is < .05, indicating that we can reject the null hypothesis and that we are 95% confident that there exists a statistically significant difference between callback rates.

5: It is not necessarily the most important factor in callback success. Even if there exists racial discrimination for applicants with similar credentials, other factors like qualifications and recent employment status would play a role in determining callback rates. I would amend the analysis to focus on different factors like years of experience and computer skills, determining if a racial difference exists at different levels of the workforce. Another concern would be which jobs the resumes were sent to, whether they were random or targeted towards postings more likely to be a fit for the resume. Since the callback rates are low for both resumes with white names and black names, this could be distorting the real callback rates experienced in the broader population of jobseekers.